# Load data in Postgres

- Used http://www.convertcsv.com/csv-to-sql.htm to generate CREATE TABLE commands by pasting 1st 100 lines from CSV
- Edited all NUMERIC fields to REAL
- Edited some VARCHAR lengths as necessary
- Created a couple of aggregate tables

In [3]:
import psycopg2

try:
    conn = psycopg2.connect("dbname='homecredit' user='postgres' host='localhost' password='metis'")
    cur = conn.cursor()
except w:
    print(w)
    print ("I am unable to connect to the database")

In [2]:
import pandas as pd

application_test = pd.read_csv("data/application_test.csv")
application_train = pd.read_csv("data/application_train.csv")
bureau = pd.read_csv("data/bureau.csv")
bureau_balance = pd.read_csv("data/bureau_balance.csv")
credit_card_balance = pd.read_csv("data/credit_card_balance.csv")
installments_payments = pd.read_csv("data/installments_payments.csv")
POS_CASH_balance = pd.read_csv("data/POS_CASH_balance.csv")
previous_application = pd.read_csv("data/previous_application.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")


In [3]:
def drop_table(table):
    global cur
    global conn
    conn.rollback()
    sqlstr = 'drop table if exists %s;' % table
    cur.execute(sqlstr)
    conn.commit()

def load_table(table_name):
    global cur
    global conn
    sqlstr = """COPY %s FROM '/home/druce/Notebooks/homecredit/data/%s.csv' DELIMITER ',' CSV HEADER;""" % (table_name, table_name)
    cur.execute(sqlstr)
    conn.commit()
    

In [4]:
list(application_test.dtypes[application_test.dtypes == 'object'].reset_index()["index"])

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [5]:
stringcols = ["NAME_CONTRACT_TYPE",
              "CODE_GENDER",
              "FLAG_OWN_CAR",
              "FLAG_OWN_REALTY",
              "NAME_TYPE_SUITE",
              "NAME_INCOME_TYPE",
              "NAME_EDUCATION_TYPE",
              "NAME_FAMILY_STATUS",
              "NAME_HOUSING_TYPE",
              "OCCUPATION_TYPE",
              "WEEKDAY_APPR_PROCESS_START",
              "ORGANIZATION_TYPE",
              "FONDKAPREMONT_MODE",
              "HOUSETYPE_MODE",
              "WALLSMATERIAL_MODE",
              "EMERGENCYSTATE_MODE",
             ]

for col in stringcols:
    application_test[col] = application_test[col].astype(str)
    print(col, application_test[col].map(len).max())

NAME_CONTRACT_TYPE 15
CODE_GENDER 1
FLAG_OWN_CAR 1
FLAG_OWN_REALTY 1
NAME_TYPE_SUITE 15
NAME_INCOME_TYPE 20
NAME_EDUCATION_TYPE 29
NAME_FAMILY_STATUS 20
NAME_HOUSING_TYPE 19
OCCUPATION_TYPE 21
WEEKDAY_APPR_PROCESS_START 9
ORGANIZATION_TYPE 22
FONDKAPREMONT_MODE 21
HOUSETYPE_MODE 16
WALLSMATERIAL_MODE 12
EMERGENCYSTATE_MODE 3


In [7]:
#application_test = pd.read_csv("data/application_test.csv")
drop_table('application_test')

sqlstr = """
CREATE TABLE application_test (
SK_ID_CURR                   INTEGER  NOT NULL PRIMARY KEY   ,
NAME_CONTRACT_TYPE           VARCHAR(15) NOT NULL  ,
CODE_GENDER                  VARCHAR(3) NOT NULL  ,
FLAG_OWN_CAR                 VARCHAR(1) NOT NULL  ,
FLAG_OWN_REALTY              VARCHAR(1) NOT NULL  ,
CNT_CHILDREN                 INTEGER  NOT NULL  ,
AMT_INCOME_TOTAL             REAL NOT NULL  ,
AMT_CREDIT                   REAL NOT NULL  ,
AMT_ANNUITY                  REAL NULL  ,
AMT_GOODS_PRICE              REAL NULL,
NAME_TYPE_SUITE              VARCHAR(15)  ,
NAME_INCOME_TYPE             VARCHAR(20) NOT NULL  ,
NAME_EDUCATION_TYPE          VARCHAR(29) NOT NULL  ,
NAME_FAMILY_STATUS           VARCHAR(20) NOT NULL  ,
NAME_HOUSING_TYPE            VARCHAR(19) NOT NULL  ,
REGION_POPULATION_RELATIVE   REAL NOT NULL  ,
DAYS_BIRTH                   INTEGER  NOT NULL  ,
DAYS_EMPLOYED                INTEGER  NOT NULL  ,
DAYS_REGISTRATION            REAL NOT NULL  ,
DAYS_ID_PUBLISH              INTEGER  NOT NULL  ,
OWN_CAR_AGE                  REAL  ,
FLAG_MOBIL                   BIT  NOT NULL  ,
FLAG_EMP_PHONE               BIT  NOT NULL  ,
FLAG_WORK_PHONE              BIT  NOT NULL  ,
FLAG_CONT_MOBILE             BIT  NOT NULL  ,
FLAG_PHONE                   BIT  NOT NULL  ,
FLAG_EMAIL                   BIT  NOT NULL  ,
OCCUPATION_TYPE              VARCHAR(21)  ,
CNT_FAM_MEMBERS              REAL NULL  ,
REGION_RATING_CLIENT         INTEGER  NOT NULL  ,
REGION_RATING_CLIENT_W_CITY  INTEGER  NOT NULL  ,
WEEKDAY_APPR_PROCESS_START   VARCHAR(9) NOT NULL  ,
HOUR_APPR_PROCESS_START      INTEGER  NOT NULL  ,
REG_REGION_NOT_LIVE_REGION   BIT  NOT NULL  ,
REG_REGION_NOT_WORK_REGION   BIT  NOT NULL  ,
LIVE_REGION_NOT_WORK_REGION  BIT  NOT NULL  ,
REG_CITY_NOT_LIVE_CITY       BIT  NOT NULL  ,
REG_CITY_NOT_WORK_CITY       BIT  NOT NULL  ,
LIVE_CITY_NOT_WORK_CITY      BIT  NOT NULL  ,
ORGANIZATION_TYPE            VARCHAR(22) NOT NULL  ,
EXT_SOURCE_1                 REAL  ,
EXT_SOURCE_2                 REAL ,
EXT_SOURCE_3                 REAL  ,
APARTMENTS_AVG               REAL  ,
BASEMENTAREA_AVG             REAL  ,
YEARS_BEGINEXPLUATATION_AVG  REAL  ,
YEARS_BUILD_AVG              REAL  ,
COMMONAREA_AVG               REAL  ,
ELEVATORS_AVG                REAL  ,
ENTRANCES_AVG                REAL  ,
FLOORSMAX_AVG                REAL  ,
FLOORSMIN_AVG                REAL  ,
LANDAREA_AVG                 REAL  ,
LIVINGAPARTMENTS_AVG         REAL  ,
LIVINGAREA_AVG               REAL  ,
NONLIVINGAPARTMENTS_AVG      REAL  ,
NONLIVINGAREA_AVG            REAL  ,
APARTMENTS_MODE              REAL  ,
BASEMENTAREA_MODE            REAL  ,
YEARS_BEGINEXPLUATATION_MODE REAL  ,
YEARS_BUILD_MODE             REAL  ,
COMMONAREA_MODE              REAL  ,
ELEVATORS_MODE               REAL  ,
ENTRANCES_MODE               REAL  ,
FLOORSMAX_MODE               REAL  ,
FLOORSMIN_MODE               REAL  ,
LANDAREA_MODE                REAL  ,
LIVINGAPARTMENTS_MODE        REAL  ,
LIVINGAREA_MODE              REAL  ,
NONLIVINGAPARTMENTS_MODE     REAL  ,
NONLIVINGAREA_MODE           REAL  ,
APARTMENTS_MEDI              REAL  ,
BASEMENTAREA_MEDI            REAL  ,
YEARS_BEGINEXPLUATATION_MEDI REAL  ,
YEARS_BUILD_MEDI             REAL  ,
COMMONAREA_MEDI              REAL  ,
ELEVATORS_MEDI               REAL  ,
ENTRANCES_MEDI               REAL  ,
FLOORSMAX_MEDI               REAL  ,
FLOORSMIN_MEDI               REAL  ,
LANDAREA_MEDI                REAL  ,
LIVINGAPARTMENTS_MEDI        REAL  ,
LIVINGAREA_MEDI              REAL  ,
NONLIVINGAPARTMENTS_MEDI     REAL  ,
NONLIVINGAREA_MEDI           REAL  ,
FONDKAPREMONT_MODE           VARCHAR(21)  ,
HOUSETYPE_MODE               VARCHAR(16)  ,
TOTALAREA_MODE               REAL  ,
WALLSMATERIAL_MODE           VARCHAR(12)  ,
EMERGENCYSTATE_MODE          VARCHAR(3)  ,
OBS_30_CNT_SOCIAL_CIRCLE     REAL   ,
DEF_30_CNT_SOCIAL_CIRCLE     REAL   ,
OBS_60_CNT_SOCIAL_CIRCLE     REAL   ,
DEF_60_CNT_SOCIAL_CIRCLE     REAL   ,
DAYS_LAST_PHONE_CHANGE       REAL   ,
FLAG_DOCUMENT_2              BIT  NOT NULL  ,
FLAG_DOCUMENT_3              BIT  NOT NULL  ,
FLAG_DOCUMENT_4              BIT  NOT NULL  ,
FLAG_DOCUMENT_5              BIT  NOT NULL  ,
FLAG_DOCUMENT_6              BIT  NOT NULL  ,
FLAG_DOCUMENT_7              BIT  NOT NULL  ,
FLAG_DOCUMENT_8              BIT  NOT NULL  ,
FLAG_DOCUMENT_9              BIT  NOT NULL  ,
FLAG_DOCUMENT_10             BIT  NOT NULL  ,
FLAG_DOCUMENT_11             BIT  NOT NULL  ,
FLAG_DOCUMENT_12             BIT  NOT NULL  ,
FLAG_DOCUMENT_13             BIT  NOT NULL  ,
FLAG_DOCUMENT_14             BIT  NOT NULL  ,
FLAG_DOCUMENT_15             BIT  NOT NULL  ,
FLAG_DOCUMENT_16             BIT  NOT NULL  ,
FLAG_DOCUMENT_17             BIT  NOT NULL  ,
FLAG_DOCUMENT_18             BIT  NOT NULL  ,
FLAG_DOCUMENT_19             BIT  NOT NULL  ,
FLAG_DOCUMENT_20             BIT  NOT NULL  ,
FLAG_DOCUMENT_21             BIT  NOT NULL  ,
AMT_REQ_CREDIT_BUREAU_HOUR   REAL  ,
AMT_REQ_CREDIT_BUREAU_DAY    REAL  ,
AMT_REQ_CREDIT_BUREAU_WEEK   REAL  ,
AMT_REQ_CREDIT_BUREAU_MON    REAL  ,
AMT_REQ_CREDIT_BUREAU_QRT    REAL  ,
AMT_REQ_CREDIT_BUREAU_YEAR   REAL);

"""

cur.execute(sqlstr)
conn.commit()

In [8]:
load_table('application_test')



In [9]:
#application_train = pd.read_csv("data/application_train.csv")
list(application_train.dtypes[application_train.dtypes == 'object'].reset_index()["index"])

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [10]:
stringcols = ["NAME_CONTRACT_TYPE",
              "CODE_GENDER",
              "FLAG_OWN_CAR",
              "FLAG_OWN_REALTY",
              "NAME_TYPE_SUITE",
              "NAME_INCOME_TYPE",
              "NAME_EDUCATION_TYPE",
              "NAME_FAMILY_STATUS",
              "NAME_HOUSING_TYPE",
              "OCCUPATION_TYPE",
              "WEEKDAY_APPR_PROCESS_START",
              "ORGANIZATION_TYPE",
              "FONDKAPREMONT_MODE",
              "HOUSETYPE_MODE",
              "WALLSMATERIAL_MODE",
              "EMERGENCYSTATE_MODE",
             ]

for col in stringcols:
    application_train[col] = application_train[col].astype(str)
    print(col, application_train[col].map(len).max())

NAME_CONTRACT_TYPE 15
CODE_GENDER 3
FLAG_OWN_CAR 1
FLAG_OWN_REALTY 1
NAME_TYPE_SUITE 15
NAME_INCOME_TYPE 20
NAME_EDUCATION_TYPE 29
NAME_FAMILY_STATUS 20
NAME_HOUSING_TYPE 19
OCCUPATION_TYPE 21
WEEKDAY_APPR_PROCESS_START 9
ORGANIZATION_TYPE 22
FONDKAPREMONT_MODE 21
HOUSETYPE_MODE 16
WALLSMATERIAL_MODE 12
EMERGENCYSTATE_MODE 3


In [11]:
drop_table('application_train')

sqlstr = """
CREATE TABLE application_train (
SK_ID_CURR                   INTEGER  NOT NULL PRIMARY KEY   ,
TARGET                       BIT  NOT NULL  ,
NAME_CONTRACT_TYPE           VARCHAR(15) NOT NULL  ,
CODE_GENDER                  VARCHAR(3) NOT NULL  ,
FLAG_OWN_CAR                 VARCHAR(1) NOT NULL  ,
FLAG_OWN_REALTY              VARCHAR(1) NOT NULL  ,
CNT_CHILDREN                 INTEGER  NOT NULL  ,
AMT_INCOME_TOTAL             REAL NOT NULL  ,
AMT_CREDIT                   REAL NOT NULL  ,
AMT_ANNUITY                  REAL NULL  ,
AMT_GOODS_PRICE              REAL NULL  ,
NAME_TYPE_SUITE              VARCHAR(15)  ,
NAME_INCOME_TYPE             VARCHAR(20) NOT NULL  ,
NAME_EDUCATION_TYPE          VARCHAR(29) NOT NULL  ,
NAME_FAMILY_STATUS           VARCHAR(20) NOT NULL  ,
NAME_HOUSING_TYPE            VARCHAR(19) NOT NULL  ,
REGION_POPULATION_RELATIVE   REAL NOT NULL  ,
DAYS_BIRTH                   INTEGER  NOT NULL  ,
DAYS_EMPLOYED                INTEGER  NOT NULL  ,
DAYS_REGISTRATION            REAL NOT NULL  ,
DAYS_ID_PUBLISH              INTEGER  NOT NULL  ,
OWN_CAR_AGE                  REAL  ,
FLAG_MOBIL                   BIT  NOT NULL  ,
FLAG_EMP_PHONE               BIT  NOT NULL  ,
FLAG_WORK_PHONE              BIT  NOT NULL  ,
FLAG_CONT_MOBILE             BIT  NOT NULL  ,
FLAG_PHONE                   BIT  NOT NULL  ,
FLAG_EMAIL                   BIT  NOT NULL  ,
OCCUPATION_TYPE              VARCHAR(21)  ,
CNT_FAM_MEMBERS              REAL  NULL  ,
REGION_RATING_CLIENT         INTEGER  NOT NULL  ,
REGION_RATING_CLIENT_W_CITY  INTEGER  NOT NULL  ,
WEEKDAY_APPR_PROCESS_START   VARCHAR(9) NOT NULL  ,
HOUR_APPR_PROCESS_START      INTEGER  NOT NULL  ,
REG_REGION_NOT_LIVE_REGION   BIT  NOT NULL  ,
REG_REGION_NOT_WORK_REGION   BIT  NOT NULL  ,
LIVE_REGION_NOT_WORK_REGION  BIT  NOT NULL  ,
REG_CITY_NOT_LIVE_CITY       BIT  NOT NULL  ,
REG_CITY_NOT_WORK_CITY       BIT  NOT NULL  ,
LIVE_CITY_NOT_WORK_CITY      BIT  NOT NULL  ,
ORGANIZATION_TYPE            VARCHAR(22) NOT NULL  ,
EXT_SOURCE_1                 REAL  ,
EXT_SOURCE_2                 REAL ,
EXT_SOURCE_3                 REAL  ,
APARTMENTS_AVG               REAL  ,
BASEMENTAREA_AVG             REAL  ,
YEARS_BEGINEXPLUATATION_AVG  REAL  ,
YEARS_BUILD_AVG              REAL  ,
COMMONAREA_AVG               REAL  ,
ELEVATORS_AVG                REAL  ,
ENTRANCES_AVG                REAL  ,
FLOORSMAX_AVG                REAL  ,
FLOORSMIN_AVG                REAL  ,
LANDAREA_AVG                 REAL  ,
LIVINGAPARTMENTS_AVG         REAL  ,
LIVINGAREA_AVG               REAL  ,
NONLIVINGAPARTMENTS_AVG      REAL  ,
NONLIVINGAREA_AVG            REAL  ,
APARTMENTS_MODE              REAL  ,
BASEMENTAREA_MODE            REAL  ,
YEARS_BEGINEXPLUATATION_MODE REAL  ,
YEARS_BUILD_MODE             REAL  ,
COMMONAREA_MODE              REAL  ,
ELEVATORS_MODE               REAL  ,
ENTRANCES_MODE               REAL  ,
FLOORSMAX_MODE               REAL  ,
FLOORSMIN_MODE               REAL  ,
LANDAREA_MODE                REAL  ,
LIVINGAPARTMENTS_MODE        REAL  ,
LIVINGAREA_MODE              REAL  ,
NONLIVINGAPARTMENTS_MODE     REAL  ,
NONLIVINGAREA_MODE           REAL  ,
APARTMENTS_MEDI              REAL  ,
BASEMENTAREA_MEDI            REAL  ,
YEARS_BEGINEXPLUATATION_MEDI REAL  ,
YEARS_BUILD_MEDI             REAL  ,
COMMONAREA_MEDI              REAL  ,
ELEVATORS_MEDI               REAL  ,
ENTRANCES_MEDI               REAL  ,
FLOORSMAX_MEDI               REAL  ,
FLOORSMIN_MEDI               REAL  ,
LANDAREA_MEDI                REAL  ,
LIVINGAPARTMENTS_MEDI        REAL  ,
LIVINGAREA_MEDI              REAL  ,
NONLIVINGAPARTMENTS_MEDI     REAL  ,
NONLIVINGAREA_MEDI           REAL  ,
FONDKAPREMONT_MODE           VARCHAR(21)  ,
HOUSETYPE_MODE               VARCHAR(16)  ,
TOTALAREA_MODE               REAL  ,
WALLSMATERIAL_MODE           VARCHAR(12)  ,
EMERGENCYSTATE_MODE          VARCHAR(3)  ,
OBS_30_CNT_SOCIAL_CIRCLE     REAL   ,
DEF_30_CNT_SOCIAL_CIRCLE     REAL   ,
OBS_60_CNT_SOCIAL_CIRCLE     REAL   ,
DEF_60_CNT_SOCIAL_CIRCLE     REAL   ,
DAYS_LAST_PHONE_CHANGE       REAL   ,
FLAG_DOCUMENT_2              BIT  NOT NULL  ,
FLAG_DOCUMENT_3              BIT  NOT NULL  ,
FLAG_DOCUMENT_4              BIT  NOT NULL  ,
FLAG_DOCUMENT_5              BIT  NOT NULL  ,
FLAG_DOCUMENT_6              BIT  NOT NULL  ,
FLAG_DOCUMENT_7              BIT  NOT NULL  ,
FLAG_DOCUMENT_8              BIT  NOT NULL  ,
FLAG_DOCUMENT_9              BIT  NOT NULL  ,
FLAG_DOCUMENT_10             BIT  NOT NULL  ,
FLAG_DOCUMENT_11             BIT  NOT NULL  ,
FLAG_DOCUMENT_12             BIT  NOT NULL  ,
FLAG_DOCUMENT_13             BIT  NOT NULL  ,
FLAG_DOCUMENT_14             BIT  NOT NULL  ,
FLAG_DOCUMENT_15             BIT  NOT NULL  ,
FLAG_DOCUMENT_16             BIT  NOT NULL  ,
FLAG_DOCUMENT_17             BIT  NOT NULL  ,
FLAG_DOCUMENT_18             BIT  NOT NULL  ,
FLAG_DOCUMENT_19             BIT  NOT NULL  ,
FLAG_DOCUMENT_20             BIT  NOT NULL  ,
FLAG_DOCUMENT_21             BIT  NOT NULL  ,
AMT_REQ_CREDIT_BUREAU_HOUR   REAL  ,
AMT_REQ_CREDIT_BUREAU_DAY    REAL  ,
AMT_REQ_CREDIT_BUREAU_WEEK   REAL  ,
AMT_REQ_CREDIT_BUREAU_MON    REAL  ,
AMT_REQ_CREDIT_BUREAU_QRT    REAL  ,
AMT_REQ_CREDIT_BUREAU_YEAR   REAL);
"""
cur.execute(sqlstr)
conn.commit()


In [12]:
load_table("application_train")


In [13]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [14]:
bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.610875e+06,1.082775e+06,5.919400e+05,1.716428e+06,1.716415e+06,1.458759e+06,1.124648e+06,1.716428e+06,1.716428e+06,4.896370e+05
mean,2.782149e+05,5.924434e+06,-1.142108e+03,8.181666e-01,5.105174e+02,-1.017437e+03,3.825418e+03,6.410406e-03,3.549946e+05,1.370851e+05,6.229515e+03,3.791276e+01,-5.937483e+02,1.571276e+04
std,1.029386e+05,5.322657e+05,7.951649e+02,3.654443e+01,4.994220e+03,7.140106e+02,2.060316e+05,9.622391e-02,1.149811e+06,6.774011e+05,4.503203e+04,5.937650e+03,7.207473e+02,3.258269e+05
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.206000e+04,-4.202300e+04,0.000000e+00,0.000000e+00,0.000000e+00,-4.705600e+06,-5.864061e+05,0.000000e+00,-4.194700e+04,0.000000e+00
25%,1.888668e+05,5.463954e+06,-1.666000e+03,0.000000e+00,-1.138000e+03,-1.489000e+03,0.000000e+00,0.000000e+00,5.130000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-9.080000e+02,0.000000e+00
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.300000e+02,-8.970000e+02,0.000000e+00,0.000000e+00,1.255185e+05,0.000000e+00,0.000000e+00,0.000000e+00,-3.950000e+02,0.000000e+00
75%,3.674260e+05,6.385681e+06,-4.740000e+02,0.000000e+00,4.740000e+02,-4.250000e+02,0.000000e+00,0.000000e+00,3.150000e+05,4.015350e+04,0.000000e+00,0.000000e+00,-3.300000e+01,1.350000e+04
max,4.562550e+05,6.843457e+06,0.000000e+00,2.792000e+03,3.119900e+04,0.000000e+00,1.159872e+08,9.000000e+00,5.850000e+08,1.701000e+08,4.705600e+06,3.756681e+06,3.720000e+02,1.184534e+08


In [15]:
bureau["CREDIT_ACTIVE"].map(len).max()

8

In [16]:
drop_table('bureau')

sqlstr = """
CREATE TABLE bureau (
  SK_ID_CURR             INTEGER  NOT NULL,
  SK_ID_BUREAU           INTEGER  NOT NULL,
  CREDIT_ACTIVE          VARCHAR(10) NOT NULL,
  CREDIT_CURRENCY        VARCHAR(12)  NOT NULL,
  DAYS_CREDIT            INTEGER  NOT NULL,
  CREDIT_DAY_OVERDUE     INTEGER  NOT NULL,
  DAYS_CREDIT_ENDDATE    REAL,
  DAYS_ENDDATE_FACT      REAL,
  AMT_CREDIT_MAX_OVERDUE REAL,
  CNT_CREDIT_PROLONG     INTEGER  NOT NULL,
  AMT_CREDIT_SUM         REAL,
  AMT_CREDIT_SUM_DEBT    REAL,
  AMT_CREDIT_SUM_LIMIT   REAL,
  AMT_CREDIT_SUM_OVERDUE REAL,
  CREDIT_TYPE            VARCHAR(60) NOT NULL,
  DAYS_CREDIT_UPDATE     INTEGER  NOT NULL,
  AMT_ANNUITY            VARCHAR(30)
);

"""
cur.execute(sqlstr)
conn.commit()


In [17]:
load_table('bureau')


In [18]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [19]:
drop_table('bureau_balance')

sqlstr = """
CREATE TABLE bureau_balance (
SK_ID_BUREAU   INTEGER  NOT NULL,
MONTHS_BALANCE INTEGER  NOT NULL,
STATUS         VARCHAR(1) NOT NULL
);
"""
cur.execute(sqlstr)
conn.commit()


In [20]:
load_table('bureau_balance')


In [21]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [22]:
credit_card_balance.describe()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
count,3.840312e+06,3.840312e+06,3.840312e+06,3.840312e+06,3.840312e+06,3.090496e+06,3.840312e+06,3.090496e+06,3.090496e+06,3.535076e+06,...,3.840312e+06,3.840312e+06,3.840312e+06,3.090496e+06,3.840312e+06,3.090496e+06,3.090496e+06,3.535076e+06,3.840312e+06,3.840312e+06
mean,1.904504e+06,2.783242e+05,-3.452192e+01,5.830016e+04,1.538080e+05,5.961325e+03,7.433388e+03,2.881696e+02,2.968805e+03,3.540204e+03,...,5.596588e+04,5.808881e+04,5.809829e+04,3.094490e-01,7.031439e-01,4.812496e-03,5.594791e-01,2.082508e+01,9.283667e+00,3.316220e-01
std,5.364695e+05,1.027045e+05,2.666775e+01,1.063070e+05,1.651457e+05,2.822569e+04,3.384608e+04,8.201989e+03,2.079689e+04,5.600154e+03,...,1.025336e+05,1.059654e+05,1.059718e+05,1.100401e+00,3.190347e+00,8.263861e-02,3.240649e+00,2.005149e+01,9.751570e+01,2.147923e+01
min,1.000018e+06,1.000060e+05,-9.600000e+01,-4.202502e+05,0.000000e+00,-6.827310e+03,-6.211620e+03,0.000000e+00,0.000000e+00,0.000000e+00,...,-4.233058e+05,-4.202502e+05,-4.202502e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.434385e+06,1.895170e+05,-5.500000e+01,0.000000e+00,4.500000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
50%,1.897122e+06,2.783960e+05,-2.800000e+01,0.000000e+00,1.125000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00
75%,2.369328e+06,3.675800e+05,-1.100000e+01,8.904669e+04,1.800000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.633911e+03,...,8.535924e+04,8.889949e+04,8.891451e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00
max,2.843496e+06,4.562500e+05,-1.000000e+00,1.505902e+06,1.350000e+06,2.115000e+06,2.287098e+06,1.529847e+06,2.239274e+06,2.028820e+05,...,1.472317e+06,1.493338e+06,1.493338e+06,5.100000e+01,1.650000e+02,1.200000e+01,1.650000e+02,1.200000e+02,3.260000e+03,3.260000e+03


In [23]:

credit_card_balance.NAME_CONTRACT_STATUS.map(len).max()

13

In [24]:
drop_table('credit_card_balance')

sqlstr = """
CREATE TABLE credit_card_balance (
  SK_ID_PREV                 INTEGER  NOT NULL, 
  SK_ID_CURR                 INTEGER  NOT NULL,
  MONTHS_BALANCE             INTEGER  NOT NULL,
  AMT_BALANCE                REAL NOT NULL,
  AMT_CREDIT_LIMIT_ACTUAL    INTEGER  NOT NULL,
  AMT_DRAWINGS_ATM_CURRENT   REAL,
  AMT_DRAWINGS_CURRENT       REAL NOT NULL,
  AMT_DRAWINGS_OTHER_CURRENT REAL,
  AMT_DRAWINGS_POS_CURRENT   REAL,
  AMT_INST_MIN_REGULARITY    REAL,
  AMT_PAYMENT_CURRENT        REAL,
  AMT_PAYMENT_TOTAL_CURRENT  REAL NOT NULL,
  AMT_RECEIVABLE_PRINCIPAL   REAL NOT NULL,
  AMT_RECIVABLE              REAL NOT NULL,
  AMT_TOTAL_RECEIVABLE       REAL NOT NULL,
  CNT_DRAWINGS_ATM_CURRENT   REAL,
  CNT_DRAWINGS_CURRENT       INTEGER  NOT NULL,
  CNT_DRAWINGS_OTHER_CURRENT REAL,
  CNT_DRAWINGS_POS_CURRENT   REAL,
  CNT_INSTALMENT_MATURE_CUM  REAL,
  NAME_CONTRACT_STATUS       VARCHAR(15) NOT NULL,
  SK_DPD                     INTEGER  NOT NULL,
  SK_DPD_DEF                 INTEGER  NOT NULL
);
"""

cur.execute(sqlstr)
conn.commit()


In [25]:
load_table('credit_card_balance')


In [26]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [27]:
installments_payments.describe()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
count,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360250e+07,1.360540e+07,1.360250e+07
mean,1.903365e+06,2.784449e+05,8.566373e-01,1.887090e+01,-1.042270e+03,-1.051114e+03,1.705091e+04,1.723822e+04
std,5.362029e+05,1.027183e+05,1.035216e+00,2.666407e+01,8.009463e+02,8.005859e+02,5.057025e+04,5.473578e+04
min,1.000001e+06,1.000010e+05,0.000000e+00,1.000000e+00,-2.922000e+03,-4.921000e+03,0.000000e+00,0.000000e+00
25%,1.434191e+06,1.896390e+05,0.000000e+00,4.000000e+00,-1.654000e+03,-1.662000e+03,4.226085e+03,3.398265e+03
50%,1.896520e+06,2.786850e+05,1.000000e+00,8.000000e+00,-8.180000e+02,-8.270000e+02,8.884080e+03,8.125515e+03
75%,2.369094e+06,3.675300e+05,1.000000e+00,1.900000e+01,-3.610000e+02,-3.700000e+02,1.671021e+04,1.610842e+04
max,2.843499e+06,4.562550e+05,1.780000e+02,2.770000e+02,-1.000000e+00,-1.000000e+00,3.771488e+06,3.771488e+06


In [28]:
drop_table('installments_payments')

sqlstr = """CREATE TABLE installments_payments (
  SK_ID_PREV             INTEGER  NOT NULL,
  SK_ID_CURR             INTEGER  NOT NULL,
  NUM_INSTALMENT_VERSION real NOT NULL,
  NUM_INSTALMENT_NUMBER  real  NOT NULL,
  DAYS_INSTALMENT        real NOT NULL,
  DAYS_ENTRY_PAYMENT     real NULL,
  AMT_INSTALMENT         real NOT NULL,
  AMT_PAYMENT            real NULL
);
"""

cur.execute(sqlstr)
conn.commit()


In [29]:
load_table('installments_payments')


In [30]:
POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [31]:
POS_CASH_balance.describe()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF
count,1.000136e+07,1.000136e+07,1.000136e+07,9.975287e+06,9.975271e+06,1.000136e+07,1.000136e+07
mean,1.903217e+06,2.784039e+05,-3.501259e+01,1.708965e+01,1.048384e+01,1.160693e+01,6.544684e-01
std,5.358465e+05,1.027637e+05,2.606657e+01,1.199506e+01,1.110906e+01,1.327140e+02,3.276249e+01
min,1.000001e+06,1.000010e+05,-9.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.434405e+06,1.895500e+05,-5.400000e+01,1.000000e+01,3.000000e+00,0.000000e+00,0.000000e+00
50%,1.896565e+06,2.786540e+05,-2.800000e+01,1.200000e+01,7.000000e+00,0.000000e+00,0.000000e+00
75%,2.368963e+06,3.674290e+05,-1.300000e+01,2.400000e+01,1.400000e+01,0.000000e+00,0.000000e+00
max,2.843499e+06,4.562550e+05,-1.000000e+00,9.200000e+01,8.500000e+01,4.231000e+03,3.595000e+03


In [32]:
drop_table('POS_CASH_BALANCE')

sqlstr = """
CREATE TABLE POS_CASH_BALANCE(
  SK_ID_PREV            INTEGER  NOT NULL, 
  SK_ID_CURR            INTEGER  NOT NULL,
  MONTHS_BALANCE        INTEGER  NOT NULL,
  CNT_INSTALMENT        REAL NULL,
  CNT_INSTALMENT_FUTURE REAL NULL,
  NAME_CONTRACT_STATUS  VARCHAR(30) NOT NULL,
  SK_DPD                INTEGER  NOT NULL,
  SK_DPD_DEF            INTEGER  NOT NULL
);
"""
cur.execute(sqlstr)
conn.commit()


In [33]:
POS_CASH_balance["NAME_CONTRACT_STATUS"].map(len).max()

21

In [34]:
load_table('POS_CASH_balance')


In [35]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
previous_application.describe()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
count,1.670214e+06,1.670214e+06,1.297979e+06,1.670214e+06,1.670213e+06,7.743700e+05,1.284699e+06,1.670214e+06,1.670214e+06,774370.000000,...,5951.000000,1.670214e+06,1.670214e+06,1.297984e+06,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000
mean,1.923089e+06,2.783572e+05,1.595512e+04,1.752339e+05,1.961140e+05,6.697402e+03,2.278473e+05,1.248418e+01,9.964675e-01,0.079637,...,0.773503,-8.806797e+02,3.139511e+02,1.605408e+01,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838,0.332570
std,5.325980e+05,1.028148e+05,1.478214e+04,2.927798e+05,3.185746e+05,2.092150e+04,3.153966e+05,3.334028e+00,5.932963e-02,0.107823,...,0.100879,7.790997e+02,7.127443e+03,1.456729e+01,88916.115834,72444.869708,106857.034789,149647.415123,153303.516729,0.471134
min,1.000001e+06,1.000010e+05,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,-0.000015,...,0.373150,-2.922000e+03,-1.000000e+00,0.000000e+00,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000,0.000000
25%,1.461857e+06,1.893290e+05,6.321780e+03,1.872000e+04,2.416050e+04,0.000000e+00,5.084100e+04,1.000000e+01,1.000000e+00,0.000000,...,0.715645,-1.300000e+03,-1.000000e+00,6.000000e+00,365243.000000,-1628.000000,-1242.000000,-1314.000000,-1270.000000,0.000000
50%,1.923110e+06,2.787145e+05,1.125000e+04,7.104600e+04,8.054100e+04,1.638000e+03,1.123200e+05,1.200000e+01,1.000000e+00,0.051605,...,0.835095,-5.810000e+02,3.000000e+00,1.200000e+01,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000,0.000000
75%,2.384280e+06,3.675140e+05,2.065842e+04,1.803600e+05,2.164185e+05,7.740000e+03,2.340000e+05,1.500000e+01,1.000000e+00,0.108909,...,0.852537,-2.800000e+02,8.200000e+01,2.400000e+01,365243.000000,-411.000000,129.000000,-74.000000,-44.000000,1.000000
max,2.845382e+06,4.562550e+05,4.180581e+05,6.905160e+06,6.905160e+06,3.060045e+06,6.905160e+06,2.300000e+01,1.000000e+00,1.000000,...,1.000000,-1.000000e+00,4.000000e+06,8.400000e+01,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000,1.000000


In [37]:
drop_table('previous_application')

sqlstr ="""

CREATE TABLE previous_application (
  SK_ID_PREV                  INTEGER  NOT NULL PRIMARY KEY ,
  SK_ID_CURR                  INTEGER  NOT NULL,
  NAME_CONTRACT_TYPE          VARCHAR(15) NOT NULL,
  AMT_ANNUITY                 REAL,
  AMT_APPLICATION             REAL NOT NULL,
  AMT_CREDIT                  REAL,
  AMT_DOWN_PAYMENT            REAL,
  AMT_GOODS_PRICE             REAL,
  WEEKDAY_APPR_PROCESS_START  VARCHAR(9) NOT NULL,
  HOUR_APPR_PROCESS_START     INTEGER  NOT NULL,
  FLAG_LAST_APPL_PER_CONTRACT VARCHAR(1) NOT NULL,
  NFLAG_LAST_APPL_IN_DAY      BIT  NOT NULL,
  RATE_DOWN_PAYMENT           REAL,
  RATE_INTEREST_PRIMARY       REAL,
  RATE_INTEREST_PRIVILEGED    REAL,
  NAME_CASH_LOAN_PURPOSE      VARCHAR(32) NOT NULL,
  NAME_CONTRACT_STATUS        VARCHAR(12) NOT NULL,
  DAYS_DECISION               INTEGER  NOT NULL,
  NAME_PAYMENT_TYPE           VARCHAR(64) NOT NULL,
  CODE_REJECT_REASON          VARCHAR(6) NOT NULL,
  NAME_TYPE_SUITE             VARCHAR(15),
  NAME_CLIENT_TYPE            VARCHAR(9) NOT NULL,
  NAME_GOODS_CATEGORY         VARCHAR(24) NOT NULL,
  NAME_PORTFOLIO              VARCHAR(5) NOT NULL,
  NAME_PRODUCT_TYPE           VARCHAR(7) NOT NULL,
  CHANNEL_TYPE                VARCHAR(32) NOT NULL,
  SELLERPLACE_AREA            INTEGER  NOT NULL,
  NAME_SELLER_INDUSTRY        VARCHAR(20) NOT NULL,
  CNT_PAYMENT                 REAL,
  NAME_YIELD_GROUP            VARCHAR(10) NOT NULL,
  PRODUCT_COMBINATION         VARCHAR(30),
  DAYS_FIRST_DRAWING          REAL,
  DAYS_FIRST_DUE              REAL,
  DAYS_LAST_DUE_1ST_VERSION   REAL,
  DAYS_LAST_DUE               REAL,
  DAYS_TERMINATION            REAL,
  NFLAG_INSURED_ON_APPROVAL   REAL
);
"""

cur.execute(sqlstr)
conn.commit()

In [38]:
previous_application["CHANNEL_TYPE"].map(len).max()
previous_application["NAME_PAYMENT_TYPE"].map(len).max()
previous_application["NAME_CASH_LOAN_PURPOSE"].map(len).max()


32

In [39]:
load_table('previous_application')


In [10]:
# create a table with only most recent application
sqlstr = """
create table dv_latest_application as
SELECT rank_filter.* 
FROM 
(select previous_application.*, 
 rank() OVER (PARTITION BY sk_id_curr ORDER BY sk_id_curr, days_decision desc, sk_id_prev) 
 from previous_application 
 where nflag_last_appl_in_day=1::bit 
 order by sk_id_curr, days_decision desc, sk_id_prev) rank_filter 
where rank=1;"""

cur.execute(sqlstr)
conn.commit()

In [12]:
# create a table with aggregates 
# count number of applications
sqlstr = """
create table dvtemp1 as select sk_id_curr, count(*) c from previous_application group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [14]:
# count by name_contract_status

sqlstr = """
create table dvtemp2 as 
select sk_id_curr, count(*) c 
from previous_application 
where name_contract_status = 'Refused'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp3 as 
select sk_id_curr, count(*) c 
from previous_application 
where name_contract_status = 'Canceled'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp4 as 
select sk_id_curr, count(*) c 
from previous_application 
where name_contract_status = 'Approved'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp5 as 
select sk_id_curr, count(*) c 
from previous_application 
where name_contract_status = 'Unused offer'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()
 


In [15]:
# count by code_reject_reason

sqlstr = """
create table dvtemp11 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'XAP'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp12 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'CLIENT'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp13 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'VERIF'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp14 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'XNA'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp15 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'LIMIT'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp16 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'SCOFR'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp17 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'SYSTEM'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp18 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'HC'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()

sqlstr = """
create table dvtemp19 as 
select sk_id_curr, count(*) c 
from previous_application 
where code_reject_reason = 'SCO'
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()



In [16]:
# bring all into one table with big outer join
sqlstr = """
create table dv_latest_application_ext as
select dv_latest_application.*,
dvtemp1.c app_count,
dvtemp2.c refused_count,
dvtemp3.c canceled_count,
dvtemp4.c approved_count,
dvtemp5.c unused_count,
dvtemp11.c XAP_count,
dvtemp12.c CLIENT_count,
dvtemp13.c VERIF_count,
dvtemp14.c XNA_count,
dvtemp15.c LIMIT_count,
dvtemp16.c SCOFR_count,
dvtemp17.c SYSTEM_count,
dvtemp18.c HC_count,
dvtemp19.c SCO_count
from dv_latest_application
left outer join dvtemp1 on dv_latest_application.sk_id_curr = dvtemp1.sk_id_curr
left outer join dvtemp2 on dv_latest_application.sk_id_curr = dvtemp2.sk_id_curr
left outer join dvtemp3 on dv_latest_application.sk_id_curr = dvtemp3.sk_id_curr
left outer join dvtemp4 on dv_latest_application.sk_id_curr = dvtemp4.sk_id_curr
left outer join dvtemp5 on dv_latest_application.sk_id_curr = dvtemp5.sk_id_curr
left outer join dvtemp11 on dv_latest_application.sk_id_curr = dvtemp11.sk_id_curr
left outer join dvtemp12 on dv_latest_application.sk_id_curr = dvtemp12.sk_id_curr
left outer join dvtemp13 on dv_latest_application.sk_id_curr = dvtemp13.sk_id_curr
left outer join dvtemp14 on dv_latest_application.sk_id_curr = dvtemp14.sk_id_curr
left outer join dvtemp15 on dv_latest_application.sk_id_curr = dvtemp15.sk_id_curr
left outer join dvtemp16 on dv_latest_application.sk_id_curr = dvtemp16.sk_id_curr
left outer join dvtemp17 on dv_latest_application.sk_id_curr = dvtemp17.sk_id_curr
left outer join dvtemp18 on dv_latest_application.sk_id_curr = dvtemp18.sk_id_curr
left outer join dvtemp19 on dv_latest_application.sk_id_curr = dvtemp19.sk_id_curr;
"""

cur.execute(sqlstr)
conn.commit()



In [17]:
sqlstr = """

drop table if exists dvtemp1;
drop table if exists dvtemp2;
drop table if exists dvtemp3;
drop table if exists dvtemp4;
drop table if exists dvtemp5;
drop table if exists dvtemp11;
drop table if exists dvtemp12;
drop table if exists dvtemp13;
drop table if exists dvtemp14;
drop table if exists dvtemp15;
drop table if exists dvtemp16;
drop table if exists dvtemp17;
drop table if exists dvtemp18;
drop table if exists dvtemp19;

"""
cur.execute(sqlstr)
conn.commit()


In [21]:
sqlstr ="""
create table dv_bureau as
select 
sk_id_curr,
count(*) cnt_bureau_reports,
sum(amt_credit_sum) amt_credit_sum, 
sum(amt_credit_sum_debt) amt_credit_sum_debt, 
sum(amt_credit_sum_limit) amt_credit_sum_limit, 
sum(amt_credit_sum_overdue) amt_credit_sum_overdue,
max(cnt_credit_prolong) cnt_credit_prolong,
max(amt_credit_max_overdue) amt_credit_max_overdue
from bureau

where credit_active = 'Active' 
group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [20]:
conn.rollback()

In [22]:
# enhance with count of bad debt rows

sqlstr = """
create table dv_bureau_ext as
select dv_bureau.*, bad_debt.c bad_debt 
from dv_bureau left outer join 
(select 
  sk_id_curr, 
  count(*) c 
  from bureau 
  where credit_active = 'Bad debt' 
  group by sk_id_curr) bad_debt 
on bad_debt.sk_id_curr = dv_bureau.sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()




In [35]:
# count bureau balance statuses

# total count
sqlstr = """
create table dvburbal1 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on 
bureau.sk_id_bureau = bureau_balance.sk_id_bureau group by sk_id_curr;

-- bad status month 0
create table dvburbal2 as
select sk_id_curr, count(*) c 
from bureau join bureau_balance on 
bureau.sk_id_bureau = bureau_balance.sk_id_bureau 
and bureau_balance.months_balance=0 
and bureau_balance.status in ('1','2','3','4','5')
group by sk_id_curr;

create table dvburbal3 as
select sk_id_curr, count(*) c 
from bureau join bureau_balance on 
bureau.sk_id_bureau = bureau_balance.sk_id_bureau 
and bureau_balance.months_balance=-1 
and bureau_balance.status in ('1','2','3','4','5')
group by sk_id_curr;

create table dvburbal4 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='0' group by sk_id_curr;

create table dvburbal5 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='1' group by sk_id_curr;

create table dvburbal6 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='2' group by sk_id_curr;

create table dvburbal7 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='3' group by sk_id_curr;

create table dvburbal8 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='4' group by sk_id_curr;

create table dvburbal9 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='5' group by sk_id_curr;

create table dvburbal10 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='C' group by sk_id_curr;

create table dvburbal11 as
select sk_id_curr, count(*) c
from bureau join bureau_balance on
bureau.sk_id_bureau = bureau_balance.sk_id_bureau and bureau_balance.status='X' group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()



In [38]:
sqlstr = """
create table dv_bureau_ext2 as
select dv_bureau_ext.*, 
dvburbal2.c burstat0,
dvburbal3.c burstat1,
dvburbal1.c burcount,
dvburbal4.c burcount0,
dvburbal5.c burcount1,
dvburbal6.c burcount2,
dvburbal7.c burcount3,
dvburbal8.c burcount4,
dvburbal9.c burcount5,
dvburbal10.c burcountC,
dvburbal11.c burcountX
from 
dv_bureau_ext
left outer join dvburbal2 on dvburbal2.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal3 on dvburbal3.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal1 on dvburbal1.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal4 on dvburbal4.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal5 on dvburbal5.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal6 on dvburbal6.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal7 on dvburbal7.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal8 on dvburbal8.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal9 on dvburbal9.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal10 on dvburbal10.sk_id_curr = dv_bureau_ext.sk_id_curr
left outer join dvburbal11 on dvburbal11.sk_id_curr = dv_bureau_ext.sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()



In [39]:
sqlstr = """

drop table if exists dvburbal1;
drop table if exists dvburbal2;
drop table if exists dvburbal3;
drop table if exists dvburbal4;
drop table if exists dvburbal5;
drop table if exists dvburbal6;
drop table if exists dvburbal7;
drop table if exists dvburbal8;
drop table if exists dvburbal9;
drop table if exists dvburbal10;
drop table if exists dvburbal11;
"""
cur.execute(sqlstr)
conn.commit()


In [40]:
sqlstr = """
create table dv_credit_card_balance as
select sk_id_curr, count(*) credit_card_count,
 sum(amt_balance) amt_balance,
 sum(amt_credit_limit_actual) amt_credit_limit_actual,
 sum(amt_drawings_atm_current) amt_drawings_atm_current,
 sum(amt_drawings_current) amt_drawings_current,
 sum(amt_drawings_other_current) amt_drawings_other_current,
 sum(amt_drawings_pos_current) amt_drawings_pos_current,
 sum(amt_inst_min_regularity) amt_inst_min_regularity,
 sum(amt_payment_current) amt_payment_current,
 sum(amt_payment_total_current) amt_payment_total_current,
 sum(amt_receivable_principal) amt_receivable_principal,
 sum(amt_recivable) amt_recivable,
 sum(amt_total_receivable) amt_total_receivable,
 sum(cnt_drawings_atm_current) cnt_drawings_atm_current,
 sum(cnt_drawings_current) cnt_drawings_current,
 sum(cnt_drawings_other_current) cnt_drawings_other_current,
 sum(cnt_drawings_pos_current) cnt_drawings_pos_current,
 sum(cnt_instalment_mature_cum) cnt_instalment_mature_cum,
 sum(sk_dpd) sk_dpd,
 sum(sk_dpd_def) sk_dpd_def
from credit_card_balance where months_balance = -1 and name_contract_status = 'Active' group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [45]:
sqlstr = """
create table dvcc1 as 
select sk_id_curr, max(sk_dpd) max_sk_dpd, max(sk_dpd_def) max_sk_dpd_def from credit_card_balance group by sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [49]:
sqlstr = """
create table dv_credit_card_balance_ext as 
select dv_credit_card_balance.*, dvcc1.max_sk_dpd, dvcc1.max_sk_dpd_def 
from dv_credit_card_balance left outer join dvcc1 on dvcc1.sk_id_curr = dv_credit_card_balance.sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [50]:
sqlstr = """
drop table if exists dvcc1;"""
cur.execute(sqlstr)
conn.commit()


In [51]:
sqlstr = """
create table dv_pos as
select sk_id_curr, 
max(cnt_instalment) cnt_instalment, 
max(cnt_instalment_future) cnt_instalment_future, 
max(sk_dpd) pos_sk_dpd_cur, 
max(sk_dpd_def) pos_sk_dpd_def 
from pos_cash_balance 
where months_balance = -1 
and name_contract_status = 'Active' 
group by sk_id_curr ;
"""
cur.execute(sqlstr)
conn.commit()



In [52]:
sqlstr = """
create table dv_pos1 as
select sk_id_curr, count(*) pos_count, 
max(sk_dpd) pos_sk_dpd_hist, 
max(sk_dpd_def) pos_sk_dpd_def_hist 
from pos_cash_balance 
group by sk_id_curr ;
"""
cur.execute(sqlstr)
conn.commit()


In [53]:
sqlstr = """
create table dv_pos_ext as
select dv_pos.*, pos_count, pos_sk_dpd_hist, pos_sk_dpd_def_hist
from dv_pos
left outer join dv_pos1 on dv_pos.sk_id_curr = dv_pos1.sk_id_curr;
"""
cur.execute(sqlstr)
conn.commit()


In [54]:
sqlstr = """
drop table if exists dv_pos1;"""
cur.execute(sqlstr)
conn.commit()